In [1]:
import numpy as np
from fxpmath import Fxp

In [14]:
class myfi(np.ndarray):
    def __new__(cls, input_array, signed=1, w=32, f=16, rounding='nearest', like=None, quantize=True): 
        ndarray = np.asarray(input_array).view(cls)   
        if isinstance(like, myfi):
            ndarray.signed = like.signed
            ndarray.w = like.w
            ndarray.f = like.f
            ndarray.rounding = like.rounding
        else:                              
            ndarray.signed = signed
            ndarray.w = w
            ndarray.f = f
            ndarray.rounding = rounding
        if quantize:
            ndarray._quantize()
        return ndarray

    def __getitem__(self, key):
        # let single element index of array return myfi with shape (1,) instead of base dtype element      
        return myfi(super().__getitem__(key),like=self)

    def _quantize(self):
        b = self * (2**self.f)
        if self.rounding == 'nearest':
            store_int = np.round(b)
        elif self.rounding == 'floor':
            store_int = np.floor(b)
        else:
            raise ValueError(f'not support rounding method {self.rounding}')
        self[...] = myfi(store_int*self.precision, like=self, quantize=False)

    @property
    def precision(self):
        return 2**-self.f

    @property
    def upper(self):
        return 2**(self.w - self.f - (1 if self.signed else 0)) - self.precision 
    @property
    def lower(self):
        return -2**(self.w-self.f-1) if self.signed else 0

    @property
    def int(self):
        return (np.asarray(self)*2**self.f).astype(int) # return normal ndarray

    def base_repr(self, base=2, frac_point=False): # return ndarray with same shape and dtype = '<Uw' where w=self.w
        if base == 2:
            add_point = lambda s: s[:-self.f] + '.' + s[-self.f:] if frac_point else s
            func = lambda i: add_point(np.binary_repr(i,width=self.w))    
        else:
            func = lambda i: np.base_repr(i,base=base)
        return np.vectorize(func)(self.int)

    @property
    def bin(self): 
        return self.base_repr(2)
    @property 
    def bin_(self):
        return self.base_repr(2,frac_point=True)
    @property
    def hex(self):
        return self.base_repr(16)
        

    #TODO: __add__ __radd__ __iadd__ sub/mul/div/ge/gr/le/etc...
    #TODO: over/underflow
    #TODO: casting


x = np.zeros((3,2,2))
y = myfi(x,1,14,3) # call __new__ and __array_finalize__
print(y.w,y.f,y.precision)
z = y[1:]
print(z.w,z.f,z.precision)
q = myfi(x,like=y)
print(q.w,q.f,q.precision)
# y[1,1,1] # call __array_finalize__
# x.view(myfi) # call __array_fianlize__

14 3 0.125
14 3 0.125
14 3 0.125


In [22]:
myfi(-7.25,1,16,8).bin

array('1111100011000000', dtype='<U16')